In [1]:
import argparse
import itertools
import pathlib

import numpy as np
import pandas as pd
import toml

In [2]:
argparser = argparse.ArgumentParser()
argparser.add_argument("--cell_type", default="all")

args = argparser.parse_args()

cell_type = args.cell_type

usage: ipykernel_launcher.py [-h] [--cell_type CELL_TYPE]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9003 --control=9001 --hb=9000 --Session.signature_scheme="hmac-sha256" --Session.key=b"863b3111-c32c-494a-8231-d57d827b0535" --shell=9002 --transport="tcp" --iopub=9004 --f=/home/lippincm/.local/share/jupyter/runtime/kernel-v2-1814153tPXhJKyknidc.json


SystemExit: 2

/home/lippincm/miniconda3/envs/Interstellar/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
# Parameters
aggregation = True
nomic = True

In [5]:
MODEL_TYPE = "regression"

In [15]:
# toml file path
TOML_PATH = pathlib.Path("../splits.toml")
# read toml file via toml
data_splits_by_treatments = toml.load(TOML_PATH)

# define the 100% test set data treatments
test_100_percent = data_splits_by_treatments["splits"]["data_splits_100"]
test_75_percent = data_splits_by_treatments["splits"]["data_splits_75"]

In [60]:
aggregate_and_nomic_path = pathlib.Path(
    f"../../../data/{cell_type}_preprocessed_sc_norm_aggregated_nomic.parquet"
).resolve(strict=True)
aggregate_path = pathlib.Path(
    f"../../../data/SHSY5Y_preprocessed_sc_norm_aggregated.parquet"
).resolve(strict=True)
data_df = pd.read_parquet(aggregate_and_nomic_path)

data_df.head()

morphology_df = pd.read_parquet(aggregate_path)

In [67]:
# get the NSU columns
nsu_cols = [col for col in data_df.columns if "NSU" in col]
nomic_df = data_df[nsu_cols]
nomic_df.loc["Metadata_Well"] = data_df["Metadata_Well"]
nomic_df.loc["oneb_Treatment_Dose_Inhibitor_Dose"] = data_df[
    "oneb_Treatment_Dose_Inhibitor_Dose"
]

/tmp/ipykernel_1815922/3573923193.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nomic_df.loc['Metadata_Well'] = data_df['Metadata_Well']
/tmp/ipykernel_1815922/3573923193.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nomic_df.loc['oneb_Treatment_Dose_Inhibitor_Dose'] = data_df['oneb_Treatment_Dose_Inhibitor_Dose']


In [43]:
# subset each column that contains metadata
metadata = data_df.filter(regex="Metadata")

# get all columns that are not metadata except for metadata_Well
data = data_df.drop(metadata.columns, axis=1)

# get the metadata_Well column
metadata_well = metadata[
    ["Metadata_Well", "oneb_Metadata_Treatment_Dose_Inhibitor_Dose"]
]

data_df = pd.merge(data, metadata_well, left_index=True, right_index=True)

In [44]:
# drop morphology metadata
morphology_df = morphology_df.drop(
    morphology_df.filter(regex="Metadata").columns, axis=1
)
morphology_df.head()

,Cytoplasm_AreaShape_Compactness,Cytoplasm_AreaShape_Eccentricity,Cytoplasm_AreaShape_Extent,Cytoplasm_AreaShape_FormFactor,Cytoplasm_AreaShape_MajorAxisLength,Cytoplasm_AreaShape_MinorAxisLength,Cytoplasm_AreaShape_Orientation,Cytoplasm_AreaShape_Zernike_0_0,Cytoplasm_AreaShape_Zernike_1_1,Cytoplasm_AreaShape_Zernike_2_0,...,Nuclei_Texture_InverseDifferenceMoment_CorrER_3_01_256,Nuclei_Texture_InverseDifferenceMoment_CorrER_3_02_256,Nuclei_Texture_InverseDifferenceMoment_CorrER_3_03_256,Nuclei_Texture_InverseDifferenceMoment_CorrMito_3_00_256,Nuclei_Texture_InverseDifferenceMoment_CorrPM_3_00_256,Nuclei_Texture_InverseDifferenceMoment_CorrPM_3_02_256,Nuclei_Texture_SumEntropy_CorrPM_3_01_256,Nuclei_Texture_SumVariance_CorrGasdermin_3_01_256,Nuclei_Texture_SumVariance_CorrMito_3_03_256,Nuclei_Texture_SumVariance_CorrPM_3_03_256
0,0.000263,0.050292,0.011215,-0.032031,0.139148,0.092653,-0.022733,-0.004550,-0.019608,0.030791,...,-0.014364,-0.022923,-0.027343,-0.037375,-0.086228,-0.087953,0.070102,-0.000259,-0.006639,-0.007727
1,-0.063223,-0.001418,0.035864,0.036794,0.037936,0.031201,-0.012884,0.028338,0.019985,0.020941,...,0.027794,0.041122,0.036051,-0.016691,-0.021993,-0.006487,0.010231,-0.014179,-0.010604,0.018682
2,-0.062009,0.001236,0.044042,0.030464,-0.002026,0.006311,0.010789,0.030538,0.022751,0.021311,...,0.003719,-0.006883,-0.021929,0.073634,-0.058678,-0.062178,0.063928,0.012029,-0.009372,-0.008545
3,-0.031699,0.047344,-0.003990,0.002975,0.115183,0.070404,-0.007908,-0.010212,-0.004997,0.067747,...,-0.083548,-0.075748,-0.079607,0.028589,-0.074989,-0.069968,0.067970,-0.012122,-0.011881,-0.008065
4,-0.045468,0.038261,0.034279,0.023820,0.163262,0.120615,-0.000391,0.018250,-0.015776,0.079127,...,-0.084309,-0.090843,-0.084131,0.101436,-0.083068,-0.083731,0.069372,0.017024,-0.017243,-0.006698


In [45]:
# define the list of the channels
channel_list = ["DNA", "Gasdermin", "ER", "Mito", "PM"]

In [46]:
# combiantions of channels
channel_combinations = []
for i in range(1, len(channel_list) + 1):
    tmp_list = list(itertools.combinations(channel_list, i))
    channel_combinations += tmp_list

In [47]:
# set up the LOO channel with recursion for dropping multiple channels


def channel_drop(df, channel):
    df = df.drop(df.filter(regex=channel).columns, axis=1)
    return df

In [48]:
# dictionary for each df to go into
results_dict = {}

In [49]:
# get all of the the channel combinations
for i in channel_combinations:
    # keep all channels and drop all channels
    if len(i) == 5:
        # keep all channels
        tmp_df = morphology_df
        # get the remaining channels for indexing purposes
        channel_list_index = [x for x in channel_list if x in i]
        channel_list_index = "_".join(channel_list_index)
        results_dict[channel_list_index] = tmp_df

        # drop all channels
        tmp = channel_drop(morphology_df, i[0])
        for j in range(1, len(i)):
            tmp = channel_drop(tmp, i[j])
        tmp_df = tmp
        # get the remaining channels for indexing purposes
        channel_list_index = "No Channels"
        results_dict[channel_list_index] = tmp_df
    # drop 4 channels
    elif len(i) == 4:
        tmp = channel_drop(morphology_df, i[0])
        for j in range(1, len(i)):
            tmp = channel_drop(tmp, i[j])
        tmp_df = tmp
        # get the remaining channels for indexing purposes
        channel_list_index = [x for x in channel_list if x not in i]
        channel_list_index = "_".join(channel_list_index)
        results_dict[channel_list_index] = tmp_df
    # drop 3 channels
    elif len(i) == 3:
        tmp = channel_drop(morphology_df, i[0])
        for j in range(1, len(i)):
            tmp = channel_drop(tmp, i[j])
        tmp_df = tmp
        # get the remaining channels for indexing purposes
        channel_list_index = [x for x in channel_list if x not in i]
        channel_list_index = "_".join(channel_list_index)
        results_dict[channel_list_index] = tmp_df
    # drop 2 channels
    elif len(i) == 2:
        tmp = channel_drop(morphology_df, i[0])
        for j in range(1, len(i)):
            tmp = channel_drop(tmp, i[j])
        tmp_df = tmp
        # get the remaining channels for indexing purposes
        channel_list_index = [x for x in channel_list if x not in i]
        channel_list_index = "_".join(channel_list_index)
        results_dict[channel_list_index] = tmp_df
    # drop 1 channel
    elif len(i) == 1:
        tmp = channel_drop(morphology_df, i[0])
        tmp_df = tmp
        # get the remaining channels for indexing purposes
        channel_list_index = [x for x in channel_list if x not in i]
        channel_list_index = "_".join(channel_list_index)
        results_dict[channel_list_index] = tmp_df
    else:
        print("channel length error")

(154, 949)
Gasdermin_ER_Mito_PM
(154, 1132)
DNA_ER_Mito_PM
(154, 1039)
DNA_Gasdermin_Mito_PM
(154, 957)
DNA_Gasdermin_ER_PM
(154, 937)
DNA_Gasdermin_ER_Mito
(154, 949)
(154, 839)
ER_Mito_PM
(154, 949)
(154, 744)
Gasdermin_Mito_PM
(154, 949)
(154, 672)
Gasdermin_ER_PM
(154, 949)
(154, 653)
Gasdermin_ER_Mito
(154, 1132)
(154, 934)
DNA_Mito_PM
(154, 1132)
(154, 844)
DNA_ER_PM
(154, 1132)
(154, 820)
DNA_ER_Mito
(154, 1039)
(154, 751)
DNA_Gasdermin_PM
(154, 1039)
(154, 738)
DNA_Gasdermin_Mito
(154, 957)
(154, 656)
DNA_Gasdermin_ER
Mito_PM
ER_PM
ER_Mito
Gasdermin_PM
Gasdermin_Mito
Gasdermin_ER
DNA_PM
DNA_Mito
DNA_ER
DNA_Gasdermin
PM
Mito
ER
Gasdermin
DNA
DNA_Gasdermin_ER_Mito_PM
No Channels


In [71]:
# set path to save
pathlib.Path(f"../indexes/{cell_type}/regression/channels").mkdir(
    parents=True, exist_ok=True
)

# loop through the dictionary and save each dataframe
for i in results_dict:
    print(i)
    print(results_dict[i].shape)
    # rename the dictionary keys
    # combine the metadata and morphology dataframes
    new_df = pd.merge(results_dict[i], metadata_well, left_index=True, right_index=True)
    # combine the cytokine dataframes
    new_df = pd.merge(new_df, nomic_df, left_index=True, right_index=True)
    # set file path
    file_path = pathlib.Path(f"../indexes/{cell_type}/regression/channels/{i}.parquet")
    # save the dataframe
    new_df.to_parquet(file_path)

Gasdermin_ER_Mito_PM
(154, 949)
DNA_ER_Mito_PM
(154, 1132)
DNA_Gasdermin_Mito_PM
(154, 1039)
DNA_Gasdermin_ER_PM
(154, 957)
DNA_Gasdermin_ER_Mito
(154, 937)
ER_Mito_PM
(154, 839)
Gasdermin_Mito_PM
(154, 744)
Gasdermin_ER_PM
(154, 672)
Gasdermin_ER_Mito
(154, 653)
DNA_Mito_PM
(154, 934)
DNA_ER_PM
(154, 844)
DNA_ER_Mito
(154, 820)
DNA_Gasdermin_PM
(154, 751)
DNA_Gasdermin_Mito
(154, 738)
DNA_Gasdermin_ER
(154, 656)
Mito_PM
(154, 648)
ER_PM
(154, 568)
ER_Mito
(154, 545)
Gasdermin_PM
(154, 473)
Gasdermin_Mito
(154, 461)
Gasdermin_ER
(154, 389)
DNA_PM
(154, 652)
DNA_Mito
(154, 635)
DNA_ER
(154, 545)
DNA_Gasdermin
(154, 463)
PM
(154, 383)
Mito
(154, 367)
ER
(154, 287)
Gasdermin
(154, 203)
DNA
(154, 366)
DNA_Gasdermin_ER_Mito_PM
(154, 1251)
No Channels
(154, 115)


In [73]:
# get the list of the dictionary keys
index_list = list(results_dict.keys())
index_list_new = []
for i in index_list:
    index_list_new.append(i + ".parquet")
# write the list to a text file
# file path
file_write_path = pathlib.Path(f"../cytokine_list/channel_splits.txt")
with open(file_write_path, "w") as f:
    for i in index_list_new:
        f.write("%s\n" % i)